# A calibrated, constrained ensemble

FaIR, like every other simple or complex climate model, is naive. It will produce projections for whatever emissions/concentrations/forcing scenario you ask it to produce projections for. It is up to the user to determine whether these projections are useful and sensible.

We are [developing a set of parameter calibrations](https://github.com/chrisroadmap/fair-calibrate) that reproduce both observed climate change since pre-industrial and assessed climate metrics such as the equilibrium climate sensitivity from the IPCC Sixth Assessement Report.

The calibrations will be continually updated, as new data for surface temperature, ocean heat content, external forcing and emissions become available. For now, we have an IPCC AR6 WG1 version (where observational constraints are generally up to somewhere in the 2014 to 2020 period), and assessments of emergent climate metrics are from the IPCC AR6 WG1 Chapter 7. We use emissions data (historical + SSP) from the Reduced Complexity Model Intercomparison Project which was compiled for IPCC AR6 WG3 Chapter 3. We also have calibration versions for replacing historical CO2 emissions by Global Carbon Project estimates. This is v1.1.0 of the `fair-calibrate` package, and can be obtained from the DOI link below.

A two-step constraining process is produced. The first step ensures that historical simulations match observed climate change to a root-mean-square error of less than 0.16°C. The second step simultaneously distribution-fits to the following assessed ranges:

- equilibrium climate sensitivity (ECS), very likely range 2-5°C, best estimate 3°C
- transient climate response (TCR), very likely range 1.2-2.4°C, best estimate 1.8°C
- global mean surface temperature change 1850-1900 to 1995-2014, very likely range 0.67-0.98°C, best estimate 0.85°C
- effective radiative forcing from aerosol-radiation interactions 1750 to 2005-2014, very likely range -0.6 to 0 W/m², best estimate -0.3 W/m²
- effective radiative forcing from aerosol-cloud interactions 1750 to 2005-2014, very likely range -1.7 to -0.3 W/m², best estimate -1.0 W/m²
- effective radiative forcing from aerosols 1750 to 2005-2014, very likely range -2.0 to -0.6 W/m², best estimate -1.3 W/m²
- ocean heat content change 1971 to 2018, likely range 329-463 ZJ, best estimate 396 ZJ
- CO2 concentrations in 2014, very likely range 396.95-398.15 ppm, best estimate 397.55 ppm
- future warming in SSP245 1995-2014 to 2081-2100, 1.24-2.59°C, best estimate 1.81°C. Note the IPCC constraint was concentration driven, in fair-calibrate v1.1.0 an emissions-driven constraint was used.

1001 posterior ensemble members are produced from an initial prior of 1.5 million.

There are many, many, many different calibration and constraining possibilities, and it depends on your purposes as to what is appropriate. If you care about the carbon cycle, you might want to constrain on TCRE and ZEC in addition, or instead of, some of the other constraints above. Not all constraints are necessarily internally consistent, and there will be some tradeoff; it is impossible to hit the above ranges perfectly. As more constraints are added, this gets harder, or will require larger prior sample sizes.

<a href="https://doi.org/10.5281/zenodo.7694879"><img src="https://zenodo.org/badge/DOI/10.5281/zenodo.7694879.svg" align="left"/></a>

## Get required imports

[pooch](https://www.fatiando.org/pooch/latest/) is a useful package that allows downloads of external datasets to your cache, meaning that you don't have to include them in Git repositories (particularly troublesome for large files) or `.gitignore` them (difficult for exact reproduciblity, and easy to forget and accidently commit a large file).

In [ ]:
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import pooch

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

### 1. Create FaIR instance

In [ ]:
f = FAIR()

### 2. Define time horizon

A lot of analysis uses 2100 as the time horizon, but 2300 is an interesting end point to see the effects of long-term climate change. We'll set 2300 as the last time bound, so the last emissions time point is 2299.5. We could even run to 2500, as the scenarios are defined that far.

In [ ]:
f.define_time(1750, 2300, 1)  # start, end, step

## Get datasets

We'll use `pooch` to retrieve the v1.1 calibration data, and external datasets of solar and volcanic forcing that were pre-prepared for AR6 work.

In [ ]:
fair_params_1_1_0_obj = pooch.retrieve(
    url = 'https://zenodo.org/record/7694879/files/calibrated_constrained_parameters.csv',
    known_hash = 'md5:9f236c43dd18a36b7b63b94e05f3caab',
)

### Configs

The constrained dataset contains 1001 ensemble members, and 47 parameters that are varied in FaIR. The parameters pertain to `climate_configs` and `species_configs` that are varied in FaIR to produce a wide range of climate responses. There are potentially hundreds of parameters that could be varied; outside of these 47, choosing something outside of a species default would either not be sensible or make little impact.

In [ ]:
df_configs = pd.read_csv(fair_params_1_1_0_obj, index_col=0)
configs = df_configs.index  # this is used as a label for the "config" axis
df_configs

## Spin up some scenarios

In this case we'll run the eight headline SSPs, and we'll bring in some external datasets of solar and volcanic forcing. We'll use the `fill_from_rcmip()` function from FaIR, so these have to use the same scenario names that appear in the RCMIP database.

In [ ]:
scenarios = [
    "ssp119",
    "ssp126",
    "ssp245",
    "ssp370",
    "ssp434",
    "ssp460",
    "ssp534-over",
    "ssp585",
]